## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/DSL

/content/drive/MyDrive/DSL


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.co

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['21세기 컴퓨팅 콘퍼런스 서울 개최',
 '"기계-인간 합쳐 \'슈퍼맨\' 될 수 있어"',
 '"우리는 인공지능(AI)이 갖는 잠재력과 힘, 영향력을 모든 사람이 쉽게 사용하고 접근할 수 있도록 민주화하길 바랍니다"',
 "피터 리(Peter Lee) 마이크로소프트연구소 총괄 부사장은 2일 서울 서대문구 연세대에서 열린 '21세기 컴퓨팅 콘퍼런스 2016'에서 AI를 향한 미래 비전을 이같이 강조했다",
 "이 행사는 올해로 18번째로, '인간과 AI가 공존하는 시대'라는 주제로 열렸다",
 '한국에서 이 행사가 개최된 것은 2007년 이후 9년 만이다',
 "컨퍼런스 기조연설하는 피터 리 부사장 (서울=연합뉴스) 윤동진 기자 = 3일 오전 서울 연세대학교 대강당에서 '인간과 AI가 공존하는 시대'를 주제로 열린 21세기 컴퓨팅 컨퍼런스 2016에서 피터 리 마이크로소프트연구소 총괄 부사장이 4차 산업혁명에서 AI가 가져올 변화에 관해 기조연설을 하고 있다",
 '2016.11.3 mon@yna.co.kr',
 '기조연설자로 나선 리 부사장은 세계적 기업이 연구에 투자하는 목표로 "세렌디피티(Serendipity)"라는 단어를 제시하며 연구개발을 꾸준히 해야 한다고 강조했다',
 "이는 '우연히 생긴 운 좋은 일', '뜻밖의 일'이란 뜻을 담고 있다",
 '지속적인 연구를 통해 많은 이들에게 생각지도 못했던 방법으로 긍정적 혜택을 줄 수 있다는 것이다',
 '어느 때보다 빠른 속도로 발전하는 AI, 머신러닝 등의 기술이 큰 변화를 가져올 것이란 관측도 내놓았다',
 '피터 리 부사장은 "컴퓨팅 기술의 발전은 구텐베르크(1452년 독일에서 활판 인쇄술로 성서를 제작해 유럽에서 인쇄업을 대중화한 인물) 시대만큼 큰 변화를 가져올 것"이라며 "많은 이가 접근하도록 민주화가 이뤄지길 바란다"고 말했다',
 "기조연설하는 피터 리 부사장 (서울=연합뉴스) 윤동진 기자 = 3일 오전 서울 연세대학교 대강당에서 '인간과 AI가 공존하는 시대'를 주제로 열린 21세기 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]
- 단어만으로는 의미를 알기 어려운 조사, 접미사, 문장부호, 숫자를 제거했다.
- 대부분의 외국어도 의미를 알기 어렵지만, AI의 경우 중요한 단어라고 생각해 남겨 두었다.

In [8]:
okt = Okt()

In [14]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.

def CustomTokenization(corpus, stop = ['Josa','Suffix','Punctuation', 'Foreign', 'Number']):
    tokenized = []
    for i, j in okt.pos(corpus, stem = True, norm = True):
        if (j in stop) & (j != 'AI'):
            continue
        tokenized.append(i)
    return tokenized

In [15]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))
corpus_tokenized[:1]

  0%|          | 0/1020 [00:00<?, ?it/s]

[['세기', '컴퓨팅', '콘퍼런스', '서울', '개최']]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]
- 제외 단어를 없애기 위해 min_count는 0으로 설정
- 성능을 직접적으로 비교하기 위해 size는 200으로 동일하게 설정

In [16]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size=200, min_count=0)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3808


In [17]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(149817, 168420)

In [22]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('인공')

[('하다', 0.9998315572738647),
 ('등', 0.9998279809951782),
 ('AI', 0.999826967716217),
 ('이', 0.9998221397399902),
 ('되다', 0.999819815158844),
 ('만들다', 0.9998195767402649),
 ('때', 0.9998190402984619),
 ('기술', 0.9998186230659485),
 ('않다', 0.9998170137405396),
 ('있다', 0.9998046159744263)]

In [23]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3808


In [24]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31436


In [25]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 5)

(83253, 84210)

In [26]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('인공')

[('인공적', 0.6599372625350952),
 ('신경망', 0.6199375987052917),
 ('Partnership', 0.5638517737388611),
 ('유기물', 0.5531776547431946),
 ('천연', 0.544263482093811),
 ('염료', 0.5416889190673828),
 ('유전체', 0.5367520451545715),
 ('미생물', 0.5243412256240845),
 ('고분자', 0.517042875289917),
 ('생체', 0.5140447616577148)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]
- 인공지능에 대한 언급이 많아 '인공'을 기준으로 사전학습 전/후 모델의 성능을 파악해 보았다.
- 사전 학습이 되지 않은 모델에서는 관련도가 높은 'AI'가 등장하기는 하지만 나머지 단어의 연관성은 매우 낮게 나타났다.
- 사전 학습이 된 모델은 AI, 인공지능이라는 단어와 관련도가 높다고 판단할 수 있는 단어만 등장했다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]
- 시각화 후 각 포인트 간의 거리가 너무 가까워 특이한 사항을 발견하지 못했다.

In [31]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3808

In [32]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [33]:
pca = PCA(random_state = 0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [36]:
# 시각화해보세요!
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        #color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [35]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        #color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [38]:
# t-SNE

iter = [500, 1000]
perp = [5, 10, 20, 30, 40, 50]
for i in iter:
  for j in perp:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = j, learning_rate = 0.05, init = 'random', n_iter = i).fit_transform(vector)

    data = go.Scatter(
      x=tsne[:,0],
      y=tsne[:,1],
      text = unique_text,
      mode='markers',
      marker=dict(
        size=5,
        #color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
    )

    layout = go.Layout(
      autosize=False,
      width=1000,
      height=1000,
      margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
      ),
    )

    fig = go.Figure(data=[data], layout=layout)
    fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




...

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!